# 01_compute_headmotion

Compute head movement and output the data for visualization

In [1]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

In [2]:
def load_sub_raw_data(bids_root, subject_idx, run_idx):
    """
    load raw meg data. 
    """
    
    if not isinstance(subject_idx, str):
        raise ValueError('subject_dix must be str')
        
    if not isinstance(run_idx, str):
        raise ValueError('run_idx must be str')
    
    subject_data_folder = pjoin(bids_root, 'sub-{}'.format(subject_idx), 'ses-movie', 'meg')
    fname = 'sub-' + subject_idx + '_ses-movie_task-movie_run-' + run_idx + '_meg.ds'
    raw_data_path = pjoin(subject_data_folder, fname)
    raw_data = mne.io.read_raw_ctf(raw_data_path, preload='True')
    
    print('total channels number is {}'.format(len(raw_data.info['chs'])))
    print('sample frequency is {} Hz'.format(raw_data.info['sfreq']))

    return raw_data

In [3]:
def extract_hpi(raw_data):
    '''
    Extract hpi data from mne raw object.
    Returns
    -------
    hpi_data : Dict
        {channel_name: hpi_value}
    '''
    picks = mne.pick_channels_regexp(raw_data.ch_names, regexp='HLC00[123][1238]...')
    hpi_data = raw_data.get_data(picks=picks)
    
    hlc_ch = []
    for i in np.arange(12):
        hlc_ch.append(raw_data.ch_names[picks[i]])
    hpi = {}
    for i in np.arange(12):
        hpi[hlc_ch[i]] = hpi_data[i]
    hpi = sorted(hpi.items(), key=lambda d: d[0], reverse=False)
    
    return hpi

In [4]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]
coord = ['nasion', 'lpa', 'rpa']

# set path
bids_root = '/nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/'
results_pth = pjoin(os.getcwd(),'tech_val_results')
if os.path.exists(results_pth) is False:
    os.mkdir(results_pth)

In [17]:
# get hpi data
head_motion = {}
upper_bound = 0
for sub in sub_list:
    
    # get runlist
    head_motion[sub] = []
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
        
    for run in run_ls:
        # get hpi data
        sub_path = BIDSPath(subject=sub, run=int(run), task='movie', session='movie', processing='preproc', root=bids_root)
        raw = mne_bids.read_raw_bids(sub_path)
        raw_data = raw.copy().crop(tmin=raw.annotations.onset[0], tmax=raw.annotations.onset[-1])
        hpi = extract_hpi(raw_data)
        
        hpi_de = []
        for chn, val in hpi:
            hpi_de.append((chn, 1000*(val-val[0])))
        
        # compute head motion
        head_mv = []
        for j in np.arange(3):
            mv = [np.sqrt(hpi_de[j*4][1][n]**2 + hpi_de[j*4+1][1][n]**2 + hpi_de[j*4+2][1][n]**2) for n in np.arange(len(hpi_de[j-1][1]))]
            head_mv.append((coord[j], mv))
            if np.max(mv)> upper_bound:
                upper_bound = np.max(mv)
        
        head_motion[sub].append(head_mv)

# convert head motion data to 1000 bins
upper_bound = np.ceil(upper_bound)
head_bins = {}
for sub in sub_list:
    head_bins[sub] = []
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
    for run in run_ls:
        
        hm_bins = []
        for j in np.arange(3):
            mv = head_motion[sub][int(run)-1][j][1]
            sub_hm_bins = pd.cut(mv, np.linspace(0, upper_bound, 1000)).value_counts().values
            hm_bins.append((coord[j], sub_hm_bins))
        
        head_bins[sub].append(hm_bins)
        

Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 557999 =      0.000 ...   929.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-02_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 557999 =      0.000 ...   929.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-01/ses

    Read 5 compensation matrices
    Range : 0 ... 557999 =      0.000 ...   929.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-02/ses-movie/meg/sub-02_ses-movie_task-movie_run-05_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-02/ses-movie/meg/sub-02_ses-movie_task-movie_run-05_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-02/ses-movie/meg/sub-02_ses-movie_task-movie_run-06_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 557999 =      0.000 ...   929.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-02/ses-movie/meg/sub-02_ses-movie_task-movie_run-06_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-0

Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-04/ses-movie/meg/sub-04_ses-movie_task-movie_run-02_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-04/ses-movie/meg/sub-04_ses-movie_task-movie_run-03_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 551999 =      0.000 ...   919.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-04/ses-movie/meg/sub-04_ses-movie_task-movie_run-03_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-04/ses-movie/meg/sub-04_ses-movie_task-movie_run-03_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-04/ses-movie/meg/sub-04_ses-movie_task-movie_run-04_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ..

Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-05/ses-movie/meg/sub-05_ses-movie_task-movie_run-08_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 431999 =      0.000 ...   719.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-05/ses-movie/meg/sub-05_ses-movie_task-movie_run-08_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-05/ses-movie/meg/sub-05_ses-movie_task-movie_run-08_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-06/ses-movie/meg/sub-06_ses-movie_task-movie_run-01_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 551999 =      0.000 ...   919.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-06/ses

    Read 5 compensation matrices
    Range : 0 ... 551999 =      0.000 ...   919.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-07/ses-movie/meg/sub-07_ses-movie_task-movie_run-05_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-07/ses-movie/meg/sub-07_ses-movie_task-movie_run-05_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-07/ses-movie/meg/sub-07_ses-movie_task-movie_run-06_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 551999 =      0.000 ...   919.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-07/ses-movie/meg/sub-07_ses-movie_task-movie_run-06_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-0

Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-09/ses-movie/meg/sub-09_ses-movie_task-movie_run-02_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-09/ses-movie/meg/sub-09_ses-movie_task-movie_run-03_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 551999 =      0.000 ...   919.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-09/ses-movie/meg/sub-09_ses-movie_task-movie_run-03_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-09/ses-movie/meg/sub-09_ses-movie_task-movie_run-03_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-09/ses-movie/meg/sub-09_ses-movie_task-movie_run-04_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ..

Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-10/ses-movie/meg/sub-10_ses-movie_task-movie_run-08_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 431999 =      0.000 ...   719.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-10/ses-movie/meg/sub-10_ses-movie_task-movie_run-08_proc-preproc_events.tsv.
Reading channel info from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-10/ses-movie/meg/sub-10_ses-movie_task-movie_run-08_proc-preproc_channels.tsv.
Opening raw data file /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-11/ses-movie/meg/sub-11_ses-movie_task-movie_run-01_proc-preproc_meg.fif...
    Read 5 compensation matrices
    Range : 0 ... 551999 =      0.000 ...   919.998 secs
Ready.
Current compensation grade : 3
Reading events from /nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/sub-11/ses

In [18]:
# save hpi data
for sub in sub_list[1:]:
    head_bins[sub].append(np.nan)
df = pd.DataFrame(head_bins, columns=sub_list, index=run_list+['09'])
df.to_pickle(pjoin(results_pth, 'head_motion.pickle'))